#### IMPORT LIBRARIES

In [44]:
import configparser
import requests
import pandas as pd
import datetime
import psycopg2

#### CONFIGURATIONS

In [45]:
try:
  # read config 
  config = configparser.ConfigParser()
  config.read('config.ini')

  # read openweather Config
  API_KEY = config['openweather']['API_KEY']
  LAT = config['openweather']['LAT']
  LON = config['openweather']['LON']
  UNITS = config['openweather']['UNITS']

  # read database password
  DATABASE_HOST = config['database']['DATABASE_HOST']
  DATABASE_USER = config['database']['DATABASE_USER']
  DATABASE_NAME = config['database']['DATABASE_NAME']
  DATABASE_PASSWORD = config['database']['DATABASE_PASSWORD']
  
except:
  print("An exception occurred in Configurations")

#### CONNECT DATABASE

In [46]:
def dbConnect():
  try:
    conn = psycopg2.connect(f"host={DATABASE_HOST} dbname={DATABASE_NAME} user={DATABASE_USER} password={DATABASE_PASSWORD}")
    return conn
  
  except:
    print("Error occurred while connecting to Database")

## CODE

#### Parsing Raw Data Function

In [47]:
def parsingRawData(rawData: dict) -> dict[dict]:
  myDict = {
    "date": datetime.datetime.fromtimestamp(rawData["dt"]),
    "city_name": rawData["name"],
    "city_id": rawData["id"],
    "country": rawData["sys"]["country"],
    "lon": rawData["coord"]["lon"],
    "lat": rawData["coord"]["lat"],
    "base": rawData["base"],
    "temp": rawData["main"]["temp"],
    "feels_like": rawData["main"]["feels_like"],
    "timezone": rawData["timezone"]
  }
  return myDict

#### Insert Data Function

In [48]:
def insertData(parsedData: dict):
    conn = dbConnect()
    try:
        cursor = conn.cursor()

        SQL = "INSERT INTO tbldata (city_id, lat, lon, temp, feels_like, city_name, country, timezone, date) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s);"
        data = (
            parsedData["city_id"],
            parsedData['lat'],
            parsedData["lon"],
            parsedData["temp"],
            parsedData["feels_like"],
            parsedData["city_name"],
            parsedData["country"],
            parsedData["timezone"],
            parsedData["date"],
            )
        cursor.execute(SQL, data)

        conn.commit()
        count = cursor.rowcount
        print(count, "Record inserted successfully into tbldata")

    except (Exception, psycopg2.Error) as error:
        print("Failed to insert record into tbldata", error)

    finally:
        # closing database connection.
        if conn:
            cursor.close()
            conn.close()
            print("PostgreSQL connection is closed")

#### API Call

In [52]:
rawData = requests.get(f"https://api.openweathermap.org/data/2.5/weather?lat={LAT}&lon={LON}&appid={API_KEY}&units={UNITS}").json()
rawData

{'coord': {'lon': 7.5906, 'lat': 50.3681},
 'weather': [{'id': 804,
   'main': 'Clouds',
   'description': 'overcast clouds',
   'icon': '04d'}],
 'base': 'stations',
 'main': {'temp': 16.41,
  'feels_like': 16.46,
  'temp_min': 15.77,
  'temp_max': 17.46,
  'pressure': 1011,
  'humidity': 90},
 'visibility': 10000,
 'wind': {'speed': 15.65, 'deg': 337},
 'clouds': {'all': 100},
 'dt': 1697126532,
 'sys': {'type': 2,
  'id': 19193,
  'country': 'DE',
  'sunrise': 1697089572,
  'sunset': 1697129156},
 'timezone': 7200,
 'id': 2874772,
 'name': 'Lützel',
 'cod': 200}

In [53]:
parsedData = parsingRawData(rawData)
parsedData

{'date': datetime.datetime(2023, 10, 12, 18, 2, 12),
 'city_name': 'Lützel',
 'city_id': 2874772,
 'country': 'DE',
 'lon': 7.5906,
 'lat': 50.3681,
 'base': 'stations',
 'temp': 16.41,
 'feels_like': 16.46,
 'timezone': 7200}

In [54]:
insertData(parsedData)

1 Record inserted successfully into tbldata
PostgreSQL connection is closed
